In [ ]:
"""
README.md
Language: Twitter
Libraries: twitter, nltk, re, csv, time, and json
API: RESTful API - tweepy

Resources: 
https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed
https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python

Steps: 
Section A: Preparing The Test Set
Step A.1: Getting the authentication credentials
Step A.2: Authenticating our Python script
Step A.3: Creating the function to build the Test set
Section B: Preparing The Training Set
Section C: Pre-processing Tweets in The Data Sets
Section D: Naive Bayes Classifier
Step D.1: Building the vocabulary
Step D.2: Matching tweets against our vocabulary
Step D.3: Building our feature vector
Step D.4: Training the classifier
Section E: Testing The Model

"""

In [ ]:
"""
Section A: Preparing The Test Set
Step A.1: Getting the authentication credentials

Subsetps
    In order to use the python-twitter API client, you first need to acquire a set of application tokens. 
    These will be your consumer_key and consumer_secret, which get passed to twitter.Api() when starting your application.
    https://python-twitter.readthedocs.io/en/latest/getting_started.html
    
1. Register Twitter application to get our own credentials (consumer and access), create your app
    https://developer.twitter.com/en/apps
2- Authenticate our Python script with the API using the credentials.
    import twitter
    api = twitter.Api(consumer_key=[consumer key],
                      consumer_secret=[consumer secret],
                      access_token_key=[access token],
                      access_token_secret=[access token secret]
3- Create function to download tweets based on a search keyword





"""

In [ ]:
import twitter
api = twitter.Api(consumer_key=[consumer key],
                  consumer_secret=[consumer secret],
                  access_token_key=[access token],
                  access_token_secret=[access token secret]